# Hito 1
Imprimimos nuestra base de datos con la información de las calles para entender cómo debemos leer los datos con un algoritmo

In [1]:
import csv
  
with open('../data/all_database.csv', mode ='r')as file:
  csvFile = csv.reader(file, delimiter=";")
  i = 1
  colums = 1
  debugString = ""
  for lines in csvFile:
    if i > 5:
        break
    endline = "\n" if i % colums == 0 else "; "
    debugString += f"{i} {lines}{endline}"
    i+=1

  print(debugString)

1 ['id', 'osm_id', 'osm_name', 'osm_meta', 'osm_source_id', 'osm_target_id', 'clazz', 'flags', 'source', 'target', 'km', 'kmh', 'cost', 'reverse_cost', 'x1', 'y1', 'x2', 'y2']
2 ['1', '4827820', 'Plaza 2 de Mayo', '', '31035122', '31035142', '15', '1', '1', '6', '0.0389729', '70', '0.0005568', '1000000', '-77.0427831', '-12.0459308', '-77.0430896', '-12.0460958']
3 ['2', '4827820', 'Plaza 2 de Mayo', '', '31035142', '31035109', '15', '1', '6', '90275', '0.0040401', '70', '5.77e-05', '1000000', '-77.0430896', '-12.0460958', '-77.0431113', '-12.0461253']
4 ['3', '4827820', 'Plaza 2 de Mayo', '', '31035109', '31035110', '15', '1', '90275', '79', '0.0182271', '70', '0.0002604', '1000000', '-77.0431113', '-12.0461253', '-77.0431753', '-12.0462768']
5 ['4', '4827820', 'Plaza 2 de Mayo', '', '31035110', '31035112', '15', '1', '79', '51668', '0.0376738', '70', '0.0005382', '1000000', '-77.0431753', '-12.0462768', '-77.0431118', '-12.0466033']



Creamos un algoritmo que pueda recorrer la información y que registre los id de los nodos origen (osm_source_id) y los nodos destino (osm_target_id). Los cuales serán los elementos principales en nuestro grafo de adyacencia

In [3]:
def buildGraph(graph, func, limit=None):
  text = ''
  i = 1
  for key in graph.keys():
    if limit:
      if i > limit:
        break

    text += f'{key};;'

    for value in graph[key]:
      text += f'{value}' + (';' if value != graph[key][-1] else '')

    text += '\n'
    i+=1
  
  func(text)
  

In [26]:
with open('../data/all_database.csv', mode ='r')as file:
  csvFile = csv.reader(file, delimiter=";")
  origins_targets = dict()

  for line in csvFile:
    originId = line[8]
    if originId not in origins_targets.keys():
      origins_targets[originId] = []
    origins_targets[originId].append(line[9])

buildGraph(origins_targets, print, 10)

source;;target
1;;6
6;;90275
90275;;79
79;;51668;3466
51668;;7;3467
7;;10;20398
10;;89757;20115
89757;;3473
3473;;3472;20129



Guardamos esa información de las intersecciones en un archivo .csv

In [27]:
f = open("streets_intersections.csv", "w")
buildGraph(origins_targets, f.write)
f.close()

Repetimos el proceso, pero esta vez queremos registrar información de cada uno de los nodos que se usarán en el grafo. Un nodo será una coordenada que probablemente pertenece a una calle, mas no es una calle. Este nodo podría representar una intersección entre calles, pero no siempre será el caso.

In [16]:
def buildNodeInfo(nodes, func, limit=None):
  text = ''
  attributes = ['node_id', 'lat', 'lon']
  i = 1
  for node in nodes:
    if limit:
      if i > limit:
        break

    for att in attributes:
      text += f'{node[att]}' + (';' if att != attributes[-1] else '')
    text += '\n'
    i+=1
  
  func(text)

In [46]:
with open('../data/all_database.csv', mode ='r')as file:
  csvFile = csv.reader(file, delimiter=";")
  node_ids = []
  i = 1

  for line in csvFile:
    if i == 1:
      node_ids.append({'node_id': 'node_id', 'lat': 'latitude', 'lon': 'longitude'})
      i+=1
      continue

    originId = line[8]
    targetId = line[9]

    canAddOrigin = True
    canAddTarget = True
    
    for node in node_ids:
      if canAddOrigin and node['node_id'] == originId:
        canAddOrigin = False
      if canAddTarget and node['node_id'] == targetId:
        canAddTarget = False
      if not canAddOrigin and not canAddTarget:
        break

    if canAddOrigin:
      node_ids.append({'node_id': originId, 'lat': line[15], 'lon': line[14]})
    if canAddTarget:
      node_ids.append({'node_id': targetId, 'lat': line[17], 'lon': line[16]})

        


    # if originId not in node_ids:
    #   node_ids.append({'node_id': originId, 'lat': line[15], 'lon': line[14]})
    # if targetId not in node_ids:
    #   node_ids.append({'node_id': targetId, 'lat': line[17], 'lon': line[16]})
    i+=1

buildNodeInfo(node_ids, print, 4)

node_id;latitude;longitude
1;-12.0459308;-77.0427831
6;-12.0460958;-77.0430896
90275;-12.0461253;-77.0431113



Guardamos esa información de los nodos en un archivo .csv

In [47]:
f = open("nodes_info.csv", "w")
buildNodeInfo(node_ids, f.write)
f.close()